# Integraciones: INE, AEMET y BOE

Este notebook muestra como usar las integraciones con APIs externas.

In [ ]:
import sys
sys.path.insert(0, '..')

import asyncio
import json
import pandas as pd

# Importar clientes
from integrations.ine import ine_client
from integrations.aemet import aemet_client
from integrations.boe import boe_client

## 1. INE - Instituto Nacional de Estadistica

El INE proporciona datos estadisticos oficiales de Espana.

In [ ]:
# Buscar operaciones estadisticas
async def buscar_operaciones_ine(query):
    return await ine_client.search_operations(query)

operaciones = await buscar_operaciones_ine("empleo")
print(f"Operaciones encontradas: {len(operaciones)}")

for op in operaciones[:5]:
    print(f"  - {op.get('Nombre')} (ID: {op.get('Id')})")

In [ ]:
# Listar tablas de una operacion
async def listar_tablas_ine(operation_id):
    return await ine_client.list_tables(operation_id)

if operaciones:
    op_id = operaciones[0].get('Id')
    tablas = await listar_tablas_ine(op_id)
    print(f"Tablas para operacion {op_id}: {len(tablas)}")
    
    for t in tablas[:5]:
        print(f"  - {t.get('Nombre')[:60]}... (ID: {t.get('Id')})")

In [ ]:
# Obtener datos de una tabla
async def obtener_datos_ine(table_id, n_last=10):
    return await ine_client.get_table_data(table_id, n_last=n_last)

if tablas:
    tabla_id = tablas[0].get('Id')
    datos = await obtener_datos_ine(tabla_id, n_last=5)
    
    print(f"Datos obtenidos: {len(datos)} registros")
    
    # Convertir a DataFrame
    df_ine = pd.DataFrame(datos[:20])
    display(df_ine.head())

## 2. AEMET - Agencia Estatal de Meteorologia

**Nota:** Requiere API key gratuita de https://opendata.aemet.es

In [ ]:
# Verificar si hay API key configurada
import os
from dotenv import load_dotenv

load_dotenv()

if os.getenv('AEMET_API_KEY'):
    print("API key de AEMET configurada")
else:
    print("ADVERTENCIA: No hay API key de AEMET configurada")
    print("Obten una gratis en: https://opendata.aemet.es/centrodedescargas/altaUsuario")
    print("Luego crea un archivo .env con: AEMET_API_KEY=tu_clave")

In [ ]:
# Listar municipios (no requiere API key para ver la estructura)
async def ejemplo_aemet():
    try:
        # Obtener prevision para Madrid (codigo 28079)
        forecast = await aemet_client.get_forecast_daily("28079")
        return forecast
    except Exception as e:
        return {"error": str(e)}

if os.getenv('AEMET_API_KEY'):
    prevision = await ejemplo_aemet()
    if isinstance(prevision, list) and prevision:
        p = prevision[0]
        print(f"Prevision para: {p.get('nombre')}")
        print(f"Provincia: {p.get('provincia')}")
        print(f"Elaborado: {p.get('elaborado')}")
    else:
        print(prevision)
else:
    print("Salta este ejemplo - configura AEMET_API_KEY primero")

In [ ]:
# Listar estaciones meteorologicas
async def listar_estaciones():
    try:
        return await aemet_client.get_stations()
    except Exception as e:
        return {"error": str(e)}

if os.getenv('AEMET_API_KEY'):
    estaciones = await listar_estaciones()
    if isinstance(estaciones, list):
        print(f"Total estaciones: {len(estaciones)}")
        df_estaciones = pd.DataFrame(estaciones[:10])
        display(df_estaciones[['indicativo', 'nombre', 'provincia']].head())
    else:
        print(estaciones)

## 3. BOE - Boletin Oficial del Estado

El BOE no requiere autenticacion.

In [ ]:
# Obtener sumario del BOE de hoy (o el mas reciente)
from datetime import datetime, timedelta

async def obtener_boe_reciente():
    # Intentar ultimos 7 dias
    for i in range(7):
        fecha = datetime.now() - timedelta(days=i)
        fecha_str = fecha.strftime("%Y%m%d")
        try:
            return await boe_client.get_summary(fecha_str), fecha_str
        except:
            continue
    return None, None

sumario, fecha = await obtener_boe_reciente()
if sumario:
    print(f"BOE del {fecha}")
    data = sumario.get('data', {}).get('sumario', {})
    meta = data.get('metadatos', {})
    print(f"Numero: {meta.get('pub_numero')}")
    print(f"Paginas: {meta.get('numero_paginas')}")

In [ ]:
# Ver secciones del BOE
if sumario:
    data = sumario.get('data', {}).get('sumario', {})
    diario = data.get('diario', [])
    
    print("Secciones del BOE:")
    for seccion in diario:
        nombre = seccion.get('sumario_nombre', 'Sin nombre')
        items = seccion.get('seccion', [])
        print(f"\n{nombre}:")
        
        for dept in items[:3]:  # Primeros 3 departamentos
            dept_nombre = dept.get('departamento', 'Sin departamento')
            print(f"  - {dept_nombre}")

In [ ]:
# Obtener un documento especifico
async def obtener_documento_boe(doc_id):
    return await boe_client.get_document(doc_id)

# Ejemplo con un documento conocido (ajustar segun fecha)
# doc = await obtener_documento_boe("BOE-A-2024-12345")
# print(json.dumps(doc, indent=2, ensure_ascii=False)[:1000])

## Resumen de APIs

| API | Autenticacion | Uso Principal |
|-----|--------------|---------------|
| INE | No | Estadisticas oficiales de Espana |
| AEMET | Si (gratis) | Datos meteorologicos |
| BOE | No | Legislacion y normativa |

## Siguiente: Webhooks

En el siguiente notebook veremos como configurar notificaciones mediante webhooks.

[05_webhooks.ipynb](05_webhooks.ipynb)